## Pandas怎么实现DataFrame的Merge

Pandas的Merge，相当于Sql的Join，将**不同的表按key关联到一个表**

### Merge的语法：

merge(left, right, how: str = 'inner', on=None, left_on=None, right_on=None, left_index: bool = False, right_index: bool = False, sort: bool = False, suffixes=('_x', '_y'), copy: bool = True, indicator: bool = False, validate=None)

* left, right: 要merge的dataframe或者有name的Series
* how：join类型，'left','right','outer','inner'
* on: join的key，left和right都需要有这个key
* left_on: left的df或者series的key
* right_on： right的df或者series的key
* left_index， right_index: 使用index而不是不同的column做join
* suffixes： 两个元素的后缀，如果列有重名，自动添加后缀，默认是('_x','_y')

本次讲解提纲：
1. 电影数据集的join实例
2. 理解merge时一对一，一对多，多对多的数量对齐关系
3. 理解left join、right join、outer join的区别
4. 如果出现非key的字段重名怎么办

### 1、电影数据集的join实例

#### 电影评分数据集

是推荐系统研究的很好的数据集
<br>位于本代码目录：./datas/movielens-1m

包含三个文件：
1. 用户对电影的评分数据 ratings.dat
2. 用户本身的信息数据 users.dat
3. 电影本身的数据 movies.dat

可以关联三个表，得到一个完整的大表


In [3]:
import pandas as pd

In [4]:
df_ratings = pd.read_csv(
    "./datas/movielens-1m/ratings.dat",
    sep = "::",
    engine="python",
    names="UserID::MovieID::Rating::Timestamp".split("::")
)

In [5]:
df_ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
df_user = pd.read_csv(
    "./datas/movielens-1m/users.dat",
    sep="::",
    engine="python",
    names="UserID::Gender::Age::Occupation::Zip-code".split("::")
)

In [8]:
df_user.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [10]:
df_movies = pd.read_csv(
    "./datas/movielens-1m/movies.dat",
    sep="::",
    engine='python',
    names="MovieID::Title::Genres".split("::")
)

In [11]:
df_movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
df_ratings_user = pd.merge(
    df_ratings, df_user, left_on="UserID", right_on="UserID", how="inner"
)

In [15]:
df_ratings_user.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067
3,1,3408,4,978300275,F,1,10,48067
4,1,2355,5,978824291,F,1,10,48067


In [19]:
df_ratings_user_movies = pd.merge(
    df_ratings_user, df_movies, left_on="MovieID", right_on="MovieID", how="inner"
)

In [20]:
df_ratings_user_movies.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


### 2、理解merge时数量的对齐关系

以下关系要正确理解：
* one-to-one: 一对一关系，关联的key都是唯一的
    - 比如（学号，姓名）merge(学号, 年龄)
    - 结果条数：1*1
* one-to-many: 一对多关系，左边唯一key，右边不唯一key
    - 比如（学号，姓名）merge(学号, [语文成绩、数学成绩、英语成绩])
    - 结果条数为：1*N
* many-to-many: 多对多关系，左边右边都不是唯一的
    - 如果（学号，[语文成绩、数学成绩、英语成绩]） merge(学号, [篮球、足球、乒乓球])
    - 结果条数：M*N

#### 2.1 one-to-one 一对一关系的merge